<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/Glove_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
from scipy import spatial
from sklearn.utils import shuffle 
from sklearn.model_selection import KFold

In [ ]:
from google.colab import  drive

drive.mount('/content/drive')

Mounted at /content/drive


#Data Import

In [ ]:
df_train = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/Data/Train/clean_all_train.csv')

df_train.head()

,Unnamed: 0,Tweets,Labels,Category
0,0,How the fuck Who the heck moved my fridge ...,0.938,anger
1,1,So my Indian Uber driver just called someone t...,0.896,anger
2,2,UK I asked for my parcel to be delivered to a...,0.896,anger
3,3,so ef whichever butt wipe pulled the fire alar...,0.896,anger
4,4,do not join they put the phone down on you ta...,0.896,anger


In [ ]:
def find_length(tweet):

  return len(tweet.split())

In [ ]:
def remove_numbers(tweet):
  tweet = re.sub('[0-9]+','',tweet)
  return tweet

In [ ]:
hash={"anger":[1,0,0,0],"sadness":[0,1,0,0],"joy":[0,0,1,0],"fear":[0,0,0,1]}
def number_category(category):
  return hash[category]

In [ ]:
df_train['Encoding'] = df_train['Category'].apply(lambda category : number_category(category))
df_train['Length'] = df_train['Tweets'].apply(lambda tweet : find_length(tweet))
df_train['Tweets'] = df_train['Tweets'].apply(lambda tweet : remove_numbers(tweet))

maxLengthOfTweet = df_train['Length'].max()


#shuffal data

In [ ]:
df_train = shuffle_efficiently(df_train)

df_train.head()

,Unnamed: 0,Tweets,Labels,Category,Encoding,Length
0,2091,â€” cannot wait She said cheerfully and g...,0.771,joy,"[0, 0, 1, 0]",8
1,3452,it is true also that he does not escape until...,0.312,sadness,"[0, 1, 0, 0]",15
2,810,and naoto nod she also like tha bands but sh...,0.229,anger,"[1, 0, 0, 0]",20
3,872,I am beyond mad that I lost track of a brown s...,0.917,fear,"[0, 0, 0, 1]",27
4,3161,my oldest cat pisses me off She is always be...,0.542,sadness,"[0, 1, 0, 0]",26


In [ ]:
def shuffle_efficiently(df_train):
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train = shuffle(df_train)
  df_train.reset_index(drop=True, inplace=True)

  df_train.head(1000)

  return df_train

#using glove word embadding

In [ ]:
embeddings_dict = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.25d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      embeddings_dict[word] = vector

In [ ]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))


#Preprocess Data

In [ ]:
def tockanizer(tweet):
  tweet = df_train['Tweets'][ind].split()
  return tweet,len(tweet)

In [ ]:
tweetslist = []
for ind in df_train.index:
  tweet,length = tockanizer(df_train['Tweets'][ind])
  tweet.extend('x'*(maxLengthOfTweet - length))
  tweetslist.append(tweet)



print(tweetslist)


[['â€”', 'cannot', 'wait', 'She', 'said', 'cheerfully', 'and', 'grinned', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], ['it', 'is', 'true', 'also', 'that', 'he', 'does', 'not', 'escape', 'until', 'almost', 'all', 'the', 'car', 'sunk', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], ['and', 'naoto', 'nod', 'she', 'also', 'like', 'tha', 'bands', 'but', 'she', 'also', 'frown', 'Will', 'the', 'bands', 'Papa', 'Roach', 'be', 'at', 'concert', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 

In [ ]:
tweetseq= [];

for tweet in tweetslist:
  temp = []
  for word in tweet:
    if word != "x" and isinstance(word , str):
      try:
        temp.append(embeddings_dict[word.lower()])
      except KeyError:
        temp.append([0]*25)
    else:
      temp.append([0]*25)
    
  tweetseq.append(temp)

In [ ]:
print(embeddings_dict['i'])

[-0.26079   0.59108   0.61622  -0.70368  -0.85159  -0.23238   1.0481
  0.066642 -0.54907   0.70047  -0.87221  -0.013954 -5.9671   -0.43106
 -0.9154    0.53744   0.57099  -0.27181  -0.84178  -0.59682   0.4516
  0.34097   0.076869  0.2284    0.2758  ]


In [ ]:
tens=tf.Variable(tweetseq)
print(tens)


<tf.Variable 'Variable:0' shape=(3613, 63, 25) dtype=float32, numpy=
array([[[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.13191 , -0.01726 , -0.86139 , ..., -0.21685 ,  0.62066 ,
         -0.28587 ],
        [-0.26671 ,  0.93121 ,  0.31878 , ...,  0.52257 ,  0.96479 ,
          0.32414 ],
        ...,
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ]],

       [[ 0.16758 ,  0.21434 , -0.093086, ...,  0.32532 ,  0.73795 ,
         -0.40859 ],
        [-0.12532 , -0.20207 , -0.12672 , ..., -0.41766 ,  0.25164 ,
         -0.055809],
        [ 0.39968 ,  0.022391, -0.63025 , ..., -0.66849 ,  0.33219 ,
         -0.32562 ],
        ...,
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
   

In [ ]:
label=tf.Variable(df_train["Encoding"])
print(label)

ValueError: ignored

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(50,dropout=0.1,recurrent_dropout=0.1))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
model.build((None,maxLengthOfTweet,25))



In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 50)                15200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 15,251
Trainable params: 15,251
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(tweetseq,labels,
                  validation_split=0.2, epochs=100)